In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date, year, monotonically_increasing_id,  avg, min, coalesce, col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, FloatType

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

25/03/10 16:12:28 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.44 instead (on interface wlp44s0f0)
25/03/10 16:12:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 16:12:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1

In [17]:
df = (
    spark.read.option("sep", ";")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("datos/Turismo.csv")
)

2

In [18]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- VISITA: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- FECHA: string (nullable = true)
 |-- SATISFACCION_INFOR: integer (nullable = true)
 |-- SATISFACCION_RECORRIDO: integer (nullable = true)
 |-- SATISFACCION_GUIA: integer (nullable = true)
 |-- SATISFACCION_GLOBAL: integer (nullable = true)
 |-- LO_MEJOR: string (nullable = true)
 |-- LO_PEOR: string (nullable = true)
 |-- OPINION: string (nullable = true)
 |-- SUGERENCIAS: string (nullable = true)



In [19]:
df.show(5)

+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| ID|              VISITA|       MES|     FECHA|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|             LO_PEOR|             OPINION|         SUGERENCIAS|
+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| 12|LEYENDAS, CASAS E...| DICIEMBRE|17/12/2023|                10|                    10|               10|                 10|        7 chimeneas |                Nada|Mejor de lo que e...|Nada, todo perfecto |
| 19|  MADRID EN FEMENINO|    AGOSTO|12/08/2023|                 8|                     8|                9|                  9|Al restaurante So...

3

In [20]:
from pyspark.sql.functions import to_date

df = df.withColumn("FECHA", to_date(df["FECHA"], "dd/MM/yyyy"))
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- VISITA: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- FECHA: date (nullable = true)
 |-- SATISFACCION_INFOR: integer (nullable = true)
 |-- SATISFACCION_RECORRIDO: integer (nullable = true)
 |-- SATISFACCION_GUIA: integer (nullable = true)
 |-- SATISFACCION_GLOBAL: integer (nullable = true)
 |-- LO_MEJOR: string (nullable = true)
 |-- LO_PEOR: string (nullable = true)
 |-- OPINION: string (nullable = true)
 |-- SUGERENCIAS: string (nullable = true)



4

In [27]:
df.orderBy("SATISFACCION_GLOBAL",ascending=False).show(3)

+---+--------------------+---------+----------+------------------+----------------------+-----------------+-------------------+--------------------+-------+--------------------+--------------------+
| ID|              VISITA|      MES|     FECHA|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|LO_PEOR|             OPINION|         SUGERENCIAS|
+---+--------------------+---------+----------+------------------+----------------------+-----------------+-------------------+--------------------+-------+--------------------+--------------------+
| 15|LA HISTORIA IMPRE...|   AGOSTO|2023-08-11|                10|                    10|               10|                 10|                NULL|   NULL|       Como esperaba|Amplia nuevas vis...|
|  3|    MADRID HISTÓRICO|    MARZO|2023-03-03|                10|                    10|               10|                 10|El conjunto está ...|   NULL|Mejor de lo que e...|                NULL|
| 20|

5

In [28]:
from pyspark.sql.functions import countDistinct

df.groupBy("MES").agg(countDistinct("VISITA").alias("num_visitas")).orderBy("num_visitas", ascending=False).show()

+--------------------+-----------+
|                 MES|num_visitas|
+--------------------+-----------+
|               JULIO|          7|
|             OCTUBRE|          6|
|               JUNIO|          6|
|              AGOSTO|          6|
|          SEPTIEMBRE|          6|
|              ENERO |          6|
|           DICIEMBRE|          5|
|           NOVIEMBRE|          5|
|                Mayo|          5|
|               ABRIL|          4|
|               MARZO|          1|
|             FEBRERO|          1|
|Creo que podrían ...|          1|
|                NULL|          0|
+--------------------+-----------+



6

In [30]:
df.filter(col("MES") == "SEPTIEMBRE").select("VISITA","SATISFACCION_GLOBAL").show()

+--------------------+-------------------+
|              VISITA|SATISFACCION_GLOBAL|
+--------------------+-------------------+
|  MADRID EN FEMENINO|                 10|
|LA LATINA Y EL MU...|                 10|
|  MADRID EN FEMENINO|                  9|
|   MADRID MONUMENTAL|                 10|
|   MADRID MONUMENTAL|                 10|
|  MADRID EN FEMENINO|                  6|
|  MADRID EN FEMENINO|                  6|
|LA HISTORIA IMPRE...|                 10|
|LA LATINA Y EL MU...|                  9|
|  MADRID EN FEMENINO|                  8|
|LA HISTORIA IMPRE...|                 10|
|   MADRID MONUMENTAL|                 10|
|  MADRID EN FEMENINO|                  8|
|HISTORIAS Y LEYEN...|                  9|
|HISTORIAS Y LEYEN...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA LATINA Y EL MU...|                 10|
+----------

7

In [36]:
df.fillna("Sin comentarios", subset="LO_PEOR").show()

+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| ID|              VISITA|       MES|     FECHA|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|             LO_PEOR|             OPINION|         SUGERENCIAS|
+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| 12|LEYENDAS, CASAS E...| DICIEMBRE|2023-12-17|                10|                    10|               10|                 10|        7 chimeneas |                Nada|Mejor de lo que e...|Nada, todo perfecto |
| 19|  MADRID EN FEMENINO|    AGOSTO|2023-08-12|                 8|                     8|                9|                  9|Al restaurante So...

8

In [41]:
from pyspark.sql.functions import udf

def satisfaccion_promedio(infor, recorrido, guia):
    return (infor + recorrido + guia)/3

calcular_satisfaccion_udf = udf(satisfaccion_promedio, DoubleType())

df8 = df.withColumn("SATISFACCION_PROMEDIO", calcular_satisfaccion_udf(df["SATISFACCION_INFOR"],df["SATISFACCION_RECORRIDO"],df["SATISFACCION_GUIA"]))
df8.show()

+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+---------------------+
| ID|              VISITA|       MES|     FECHA|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|             LO_PEOR|             OPINION|         SUGERENCIAS|SATISFACCION_PROMEDIO|
+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+---------------------+
| 12|LEYENDAS, CASAS E...| DICIEMBRE|2023-12-17|                10|                    10|               10|                 10|        7 chimeneas |                Nada|Mejor de lo que e...|Nada, todo perfecto |                 10.0|
| 19|  MADRID EN FEMENINO|    AGOSTO|2023-08-12|            

In [44]:
df.createOrReplaceTempView("turismo")
dfSQL = spark.sql("SELECT *, (SATISFACCION_INFOR,SATISFACCION_RECORRIDO,SATISFACCION_GUIA)/3 AS SATISFACCION_PROMEDIO FROM turismo")
dfSQL.show()

AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(named_struct(SATISFACCION_INFOR, SATISFACCION_INFOR, SATISFACCION_RECORRIDO, SATISFACCION_RECORRIDO, SATISFACCION_GUIA, SATISFACCION_GUIA) / 3)" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("STRUCT<SATISFACCION_INFOR: INT, SATISFACCION_RECORRIDO: INT, SATISFACCION_GUIA: INT>" and "INT").; line 1 pos 10;
'Project [ID#470, VISITA#471, MES#472, FECHA#556, SATISFACCION_INFOR#474, SATISFACCION_RECORRIDO#475, SATISFACCION_GUIA#476, SATISFACCION_GLOBAL#477, LO_MEJOR#478, LO_PEOR#479, OPINION#480, SUGERENCIAS#481, (named_struct(SATISFACCION_INFOR, SATISFACCION_INFOR#474, SATISFACCION_RECORRIDO, SATISFACCION_RECORRIDO#475, SATISFACCION_GUIA, SATISFACCION_GUIA#476) / 3) AS SATISFACCION_PROMEDIO#1376]
+- SubqueryAlias turismo
   +- View (`turismo`, [ID#470,VISITA#471,MES#472,FECHA#556,SATISFACCION_INFOR#474,SATISFACCION_RECORRIDO#475,SATISFACCION_GUIA#476,SATISFAC

In [ ]:
df.write.csv("datos/resultado1.csv", overwrite = "yes") #Set mode as "overwrite" to overwrite the existing path.


TypeError: DataFrameWriter.csv() got an unexpected keyword argument 'overwrite'